In [13]:
import numpy as np
import pickle

### GloVe Embedding 

In [24]:
!sh src/glove/build_vocab.sh
!sh src/glove/cut_vocab.sh

In [25]:
!python src/glove/pickle_vocab.py
!python src/glove/cooc.py
!python src/glove/glove_solution.py

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
summing duplicates (this can take a while)
loading cooccurrence matrix
8583463 nonzero entries
using nmax = 100 , cooc.max() = 207302
initializing embeddings
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [14]:
# Load the word embeddings
embedding_matrix = np.load('manipulated/glove_embeddings.npy')
# Load the vocabulary
with open('manipulated/vocab.pkl', 'rb') as f:
    vocabulary = pickle.load(f)
# Create a dictionary to map words to their embeddings
embeddings_dict = {word: embedding_matrix[index] for index, word in enumerate(vocabulary)}

### After embedding: Load the tweets into vectors


In [15]:
### After generating the embeddings using different method

def preprocess(tweet):
    # Assuming tweets are already tokenized and separated by spaces
    return tweet.lower().split()

def tweet_to_vector(tweet, embeddings_dict):
    words = preprocess(tweet)
    word_vectors = [embeddings_dict[word] for word in words if word in embeddings_dict]
    
    # Handle the case where tweet has no valid words found in embeddings
    if not word_vectors:
        return np.zeros(next(iter(embeddings_dict.values())).shape)
    
    # Compute the average vector
    tweet_vector = np.mean(word_vectors, axis=0)
    return tweet_vector

def process_tweets(tweet_path, embeddings_dict):
    with open(tweet_path, 'r') as f:
        tweets = f.readlines()
    tweets = [tweet.rstrip('\n') for tweet in tweets]
    tweet_vectors = np.array([tweet_to_vector(tweet, embeddings_dict) for tweet in tweets])
    return tweet_vectors

In [18]:
pos_path = 'data/twitter-datasets/train_pos.txt'
neg_path = 'data/twitter-datasets/train_neg.txt'


pos_vectors = process_tweets(pos_path, embeddings_dict)
print(pos_vectors.shape)

neg_vectors = process_tweets(neg_path, embeddings_dict)
print(neg_vectors.shape)

(100000, 20)
(100000, 20)
